# 09. Filter OSM to Local Area

Create a geojson file describing the shape of a local area by town name, based on a larger geojson file of all local area boundaries provided by the government.

Then, build "osmium" command lines that can be used to filter a larger OpenStreetMap extract (XML or PBF format) down to a smaller XML area for just the target Local Area, plus a second XML extract containing the a slightly larger bounding box around the Local Area.  The second extract will be used later to capture intersections at the very margin of the first extract, which would otherwise not be detected due to the intersecting street being clipped out of the first extract.

## Configuration

Any configuration that is required to run this notebook can be customized in the next cell

In [1]:
# The name of the target area
locality = 'Mount Eliza'
#locality = 'Langwarrin'

# The name of the Government-issued local boundary geojson file
# This is expected to be found in the 'data_sources' folder
input_geojson_filename = 'LGA_boundaries_VIC.geojson'

# The name of the key that is used to filter the main geojson file down to a specific area
# This will be 'vic_loca_2' for towns or suburbs.  Examine the original file for further options.
filter_key = 'vic_loca_2'

## Code

In [2]:
# General imports
import os
import sys

# Make sure local modules can be imported
module_path_root = os.path.abspath(os.pardir)
if module_path_root not in sys.path:
    sys.path.append(module_path_root)
    
# Import local modules
import osm_gsv_utils.osm_filter as osm_filter

In [3]:
# Derived paths

# Full path to the original, government-issued file
input_geojson_path = os.path.join(module_path_root, 'data_sources', input_geojson_filename)

# A version of the locality name with spaces replaced by underscores
locality_clean   = locality.replace(' ', '_')

# Paths to both the main OpenStreetMap XML extract and a second extract that allows a wider margin
output_filename_main    = os.path.join(os.path.abspath(os.pardir), 'data_sources', 'Locality_' + locality_clean + '.osm')
output_filename_margin  = os.path.join(os.path.abspath(os.pardir), 'data_sources', 'Locality_' + locality_clean + '_margin.osm')

## Generating geojson files

This section will automatically create the required geojson files to describe the shape of the area

In [4]:
# Load the government-issued localities data into memory
vic_localities = osm_filter(input_geojson_path)

# Filter the data down to the required area, and save as a pair of geojson files
vic_localities.save_locality(locality_clean, output_filename_main, output_filename_margin, locality)

Bounding box excl margin: -38.224989, 145.061002 => -38.162589, 145.128750
Bounding box with margin: -38.226263, 145.059387 => -38.161315, 145.130364

Run the following two osmium commands:

First command:  Extract OSM data according to the official "shape" of the Locality

osmium extract --polygon=Locality_Mount_Eliza.geojson australia-latest.osm.pbf -o Locality_Mount_Eliza.osm

Second command:  Extract OSM data for a bounding box 200 meters bigger than the "shape"
This second extract will be used to ensure we do not miss any intersections with streets that are JUST outside the locality

osmium extract --polygon=Locality_Mount_Eliza_margin.geojson australia-latest.osm.pbf -o Locality_Mount_Eliza_margin.osm


# Using osmium

Now that you have geojson files, you must install and use the "osmium" tool that will allow you to filter the OpenStreetMap data down to the required extracts, using the geojson files created above.

Obtain "osmium" from here: (https://osmcode.org/osmium-tool/)

On MacOS, install "homebrew" (if not already installed) via https://brew.sh/
Then run "brew install osmium-tool"

On Windows, the easiest way to install is probably to install "Windows Subsystem for Linux" with an Ubuntu distribution, then install "osmium-tool" as per the instructions for Ubuntu.

On Ubuntu linux, run "sudo apt-get install osmium-tool"

Once the tool is installed, manually run the two "osmium extract" commands listed in the results of the above cell.  They assume that your current working directory is the 'data_sources' directory where the original OpenStreetMap extract and the geojson files you just created are.  They also assume that you have downloaded the full .osm.pbf file for Australia from http://download.geofabrik.de and saved it under its default name "australia-latest.osm.pbf".  If you're working from a different OpenStreetMap extract, please substitute its filename.